In [150]:
# Handle table-like data and matrices
import numpy as np
import pandas as pd

# Helpers
import sys
from datetime import datetime
import pickle

from sklearn.model_selection import cross_val_score,StratifiedKFold
from sklearn.metrics import accuracy_score,roc_auc_score,roc_curve,log_loss
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,cross_validate
from sklearn.metrics import classification_report
from sklearn.model_selection import LeaveOneOut,KFold,StratifiedKFold
from sklearn.decomposition import PCA,KernelPCA
import lightgbm as lgb

import sys
sys.path.insert(0,'../../') 
from helpers import *
from MAP_estimator import MAP_estimator

# Algorithm
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

data_path = '../data/'

In [151]:
count_df = pd.read_pickle(data_path+'count_df_with_label.pkl')
bin_df = pd.read_pickle(data_path+'bin_df.pkl')
freq_df = pd.read_pickle(data_path+'freq_df.pkl')
train_y = pd.read_csv(data_path+'train_y.csv')

In [152]:
count_df.head()

,sample_name,"(CAISEDSSYNSPLHF, TCRBV10, TCRBV10-03, 01, TCRBJ01, TCRBJ01-06, 01)","(CAISESTGNTEAFF, TCRBV10, TCRBV10-03, 01, TCRBJ01, TCRBJ01-01, 01)","(CAISESVSGGNNEQFF, TCRBV10, TCRBV10-03, 01, TCRBJ02, TCRBJ02-01, 01)","(CAISGDSYNEQFF, TCRBV10, TCRBV10-03, 01, TCRBJ02, TCRBJ02-01, 01)","(CAISQGMNTEAFF, TCRBV10, TCRBV10-03, 01, TCRBJ01, TCRBJ01-01, 01)","(CASGGNTIYF, TCRBV02, TCRBV02-01, 01, TCRBJ01, TCRBJ01-03, 01)","(CASGGSSYEQYF, TCRBV27, TCRBV27-01, 01, TCRBJ02, TCRBJ02-07, 01)","(CASGGVEAFF, TCRBV02, TCRBV02-01, 01, TCRBJ01, TCRBJ01-01, 01)","(CASGLSGTEAFF, TCRBV12, TCRBV12-05, 01, TCRBJ01, TCRBJ01-01, 01)",...,"(CASSSGTNNEQFF, TCRBV12, null, null, TCRBJ02, TCRBJ02-01, 01)","(CASSVDNTGELFF, TCRBV06, TCRBV06-04, null, TCRBJ02, TCRBJ02-02, 01)","(CASSVVSGGTEAFF, TCRBV09, TCRBV09-01, null, TCRBJ01, TCRBJ01-01, 01)","(CATSRLAGGYNEQFF, TCRBV15, TCRBV15-01, 01, TCRBJ02, TCRBJ02-01, 01)","(CSARDLAASSYNEQFF, TCRBV20, null, null, TCRBJ02, TCRBJ02-01, 01)","(CSARDLSSSYEQYF, TCRBV20, null, null, TCRBJ02, TCRBJ02-07, 01)","(CSARDRPYTGELFF, TCRBV20, null, null, TCRBJ02, TCRBJ02-02, 01)","(CSARGLAGGPNEQFF, TCRBV20, null, null, TCRBJ02, TCRBJ02-01, 01)","(CSARTGLGNTIYF, TCRBV20, null, null, TCRBJ01, TCRBJ01-03, 01)",phenotype_status
0,RA47,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
1,HC9,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,RA29,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,RA8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,RA63,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [87]:
# c = count_df.columns.values.tolist()[1]
# count_df.drop(c,inplace=True,axis=1)
# count_df = count_df.merge(train_y,on='sample_name')
# bin_df = bin_df.merge(train_y,on='sample_name')
# freq_df = freq_df.merge(train_y,on='sample_name')
# count_df.to_pickle('../data/count_df_with_label.pkl')

In [153]:
TCRs = count_df.drop(['sample_name','phenotype_status'],axis=1).columns.values
len(TCRs)

33768

In [154]:
clf = lgb.LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                        learning_rate=0.1, max_depth=-1, min_child_samples=10,
                        min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
                        n_jobs=-1, num_leaves=32, objective='binary', random_state=0,
                        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
                        subsample_for_bin=200000, subsample_freq=1)

In [ ]:
#     for tcr in asso_TCRs:
#         if tcr not in features:
#             features[tcr] = 1
#         else:
#             features[tcr] += 1
#     T = pd.DataFrame(features,index=[0]).T.sort_values(by=0, ascending=False)
#     asso_TCRs = T.iloc[:50].T.columns.values.tolist()

In [ ]:
# # Init lists

# auroc = []
# kf_outer = StratifiedKFold(n_splits=5)
# kf_inner = StratifiedKFold(n_splits=5)
# for train_index,test_index in kf_outer.split(count_df,count_df['phenotype_status']):
#     train_cv, test_cv = count_df.iloc[train_index], count_df.iloc[test_index]
#     auroc2 = []
#     features = {}
#     for train_index2,test_index2 in kf_inner.split(train_cv,train_cv['phenotype_status']):
#         train_cv2, test_cv2 = train_cv.iloc[train_index2], count_df.iloc[test_index2]
#         train_neg = train_cv2[train_cv2['phenotype_status']==0]
#         train_pos = train_cv2[train_cv2['phenotype_status']==1]
#         asso_TCRs = TCRs_selection_Fisher(train_neg,train_pos,TCRs,0.2)
#         for tcr in asso_TCRs:
#             if tcr not in features:
#                 features[tcr] = 1
#             else:
#                 features[tcr] += 1
#         T = pd.DataFrame(features,index=[0]).T.sort_values(by=0, ascending=False)
        
#         clf = lgb.LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
#                         learning_rate=0.1, max_depth=-1, min_child_samples=2,
#                         min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
#                         n_jobs=-1, num_leaves=31, objective='binary', random_state=0,
#                         reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
#                         subsample_for_bin=200000, subsample_freq=1)

#         clf.fit(train_cv2[asso_TCRs],train_cv2['phenotype_status'])
#         test_prob2 = clf.predict_proba(test_cv2[asso_TCRs])[:,1]
#         auc = roc_auc_score(test_cv2['phenotype_status'],test_prob2)
#         auroc2.append(auc)
#     print(auroc2)
#     asso_TCRs = T.iloc[:100].T.columns.values.tolist()
#     clf.fit(train_cv[asso_TCRs],train_cv['phenotype_status'])
#     test_prob = clf.predict_proba(test_cv[asso_TCRs])[:,1]
#     print(roc_auc_score(test_cv['phenotype_status'],test_prob))
#     break
# # print('cv auroc:',roc_auc_score(y_true,y_proba))
# # print('cv log_loss:',log_loss(y_true,y_proba))

In [47]:
# Init lists
# auroc = []
# kf = StratifiedKFold(n_splits=10)
# for train_index,test_index in kf.split(count_df,count_df['phenotype_status']):
#     train_cv, test_cv = count_df.iloc[train_index], count_df.iloc[test_index]
#     train_neg = train_cv[train_cv['phenotype_status']==0]
#     train_pos = train_cv[train_cv['phenotype_status']==1]
#     asso_TCRs = TCRs_selection_Fisher(train_neg,train_pos,TCRs,0.2)
#     features = {}
#     for tcr in asso_TCRs:
#         if tcr not in features:
#             features[tcr] = 1
#         else:
#             features[tcr] += 1
#     T = pd.DataFrame(features,index=[0]).T.sort_values(by=0, ascending=False)
#     asso_TCRs = T.iloc[:100].T.columns.values.tolist()
#     clf.fit(train_cv[asso_TCRs],train_cv['phenotype_status'])
#     test_prob = clf.predict_proba(test_cv[asso_TCRs])[:,1]
#     auc = roc_auc_score(test_cv['phenotype_status'],test_prob)
#     auroc.append(auc)

In [ ]:
# # Init lists
# features = {}

# kf1 = StratifiedKFold(n_splits=5)
# for train_index,test_index in kf.split(count_df,count_df['sample_name']):
#     train_cv, test_cv = data.iloc[train_index], data.iloc[test_index]
#     kf2 = StratifiedKFold(n_splits=5):
#         train_cv, test_cv = count_df.iloc[train_index], count_df.iloc[test_index]
#         train_neg = train_cv[train_cv['phenotype_status']==0]
#         train_pos = train_cv[train_cv['phenotype_status']==1]
#         asso_TCRs = TCRs_selection_Fisher(train_neg,train_pos,TCRs,0.2)
#         for tcr in asso_TCRs:
#             if tcr not in features:
#                 features[tcr] = 1
#             else:
#                 features[tcr] += 1

# print('cv auroc:',roc_auc_score(y_true,y_proba))
# print('cv log_loss:',log_loss(y_true,y_proba))

In [147]:
model = LogisticRegression(penalty='l1')
# results = cross_val_score(model, count_df[TCRs], count_df['phenotype_status'], cv=10,scoring='roc_auc')

In [128]:
# Init lists
sample_name = []
y_true = []
y_pred = []
y_proba = [] 

TCRs = count_df.drop(['sample_name','phenotype_status'],axis=1).columns.values

kf = LeaveOneOut()
kf2 = StratifiedKFold(10)
for train_index,test_index in kf2.split(count_df):
    data = count_df.copy(deep=True)
    train_cv, test_cv = data.iloc[train_index], data.iloc[test_index]
#     print(test_cv['sample_name'].tolist()[0])
    
    model.fit(train_cv[TCRs],train_cv['phenotype_status'])
    test_pred = model.predict(test_cv[TCRs])
    test_prob = model.predict_proba(test_cv[TCRs])[:,1]

#     sample_name.append(test_cv['sample_name'].tolist()[0])
#     y_true.append(test_cv['phenotype_status'].values[0])
#     y_pred.append(test_pred)
#     y_proba.append(test_prob)

    
    print('y_true:',test_cv['phenotype_status'].values[0],'y_pred:',test_pred,'posterior_c1:',test_prob)
    print()

    print('cv auroc:',roc_auc_score(y_true,y_proba))
    print('cv log_loss:',log_loss(y_true,y_proba))

RA47
y_true: 1 y_pred: 1 posterior_c1: 0.9234102081705327

HC9
y_true: 0 y_pred: 1 posterior_c1: 0.9575904973051389

RA29
y_true: 1 y_pred: 1 posterior_c1: 0.8594163048485612

RA8
y_true: 1 y_pred: 1 posterior_c1: 0.9578894961787621

RA63
y_true: 1 y_pred: 1 posterior_c1: 0.952001116109492

RA33
y_true: 1 y_pred: 1 posterior_c1: 0.9639505603977044

HC17
y_true: 0 y_pred: 1 posterior_c1: 0.6834269195600632

HC3
y_true: 0 y_pred: 1 posterior_c1: 0.9479667861613961

RA17
y_true: 1 y_pred: 1 posterior_c1: 0.9496664099184644

RA46
y_true: 1 y_pred: 1 posterior_c1: 0.9504410904413557

HC6
y_true: 0 y_pred: 1 posterior_c1: 0.9293392860467596

RA26
y_true: 1 y_pred: 1 posterior_c1: 0.8702485237808385

RA15
y_true: 1 y_pred: 1 posterior_c1: 0.922820189429371

RA53
y_true: 1 y_pred: 1 posterior_c1: 0.7083754770270722

RA32
y_true: 1 y_pred: 1 posterior_c1: 0.8367164598150008

RA40
y_true: 1 y_pred: 1 posterior_c1: 0.9495381840070165

HC8
y_true: 0 y_pred: 1 posterior_c1: 0.9698607964081001

RA51

In [83]:
# Init lists
sample_name = []
y_true = []
y_pred = []
y_proba = [] 

TCRs = count_df.drop(['sample_name','phenotype_status'],axis=1).columns.values

kf = LeaveOneOut()
for train_index,test_index in kf.split(count_df):
    data = count_df.copy(deep=True)
    train_cv, test_cv = data.iloc[train_index], data.iloc[test_index]
    print(test_cv['sample_name'].tolist()[0])
    
    clf.fit(train_cv[TCRs],train_cv['phenotype_status'])
    test_pred = clf.predict(test_cv[TCRs])[0]
    test_prob = clf.predict_proba(test_cv[TCRs])[:,1][0]

    sample_name.append(test_cv['sample_name'].tolist()[0])
    y_true.append(test_cv['phenotype_status'].values[0])
    y_pred.append(test_pred)
    y_proba.append(test_prob)

    
    print('y_true:',test_cv['phenotype_status'].values[0],'y_pred:',test_pred,'posterior_c1:',test_prob)
    print()

print('cv auroc:',roc_auc_score(y_true,y_proba))
print('cv log_loss:',log_loss(y_true,y_proba))

RA47
y_true: 1 y_pred: 1 posterior_c1: 0.759482647576538

HC9
y_true: 0 y_pred: 1 posterior_c1: 0.9971423534352811

RA29
y_true: 1 y_pred: 1 posterior_c1: 0.9200520455767792

RA8
y_true: 1 y_pred: 1 posterior_c1: 0.9990813573092383

RA63
y_true: 1 y_pred: 1 posterior_c1: 0.9982863089969107

RA33
y_true: 1 y_pred: 1 posterior_c1: 0.9995996603069041

HC17
y_true: 0 y_pred: 1 posterior_c1: 0.9782578460488771

HC3
y_true: 0 y_pred: 1 posterior_c1: 0.98938119308624

RA17
y_true: 1 y_pred: 1 posterior_c1: 0.999480877646686

RA46
y_true: 1 y_pred: 1 posterior_c1: 0.9672328113679446

HC6
y_true: 0 y_pred: 0 posterior_c1: 0.4633488368982644

RA26
y_true: 1 y_pred: 1 posterior_c1: 0.9248864947433011

RA15
y_true: 1 y_pred: 1 posterior_c1: 0.9857643470384974

RA53
y_true: 1 y_pred: 1 posterior_c1: 0.973145003366494

RA32
y_true: 1 y_pred: 0 posterior_c1: 0.10046917532377679

RA40
y_true: 1 y_pred: 1 posterior_c1: 0.9675427428858607

HC8
y_true: 0 y_pred: 1 posterior_c1: 0.9652885361186739

RA51
y

In [129]:
# Init lists
sample_name = []
y_true = []
y_pred = []
y_proba = [] 

TCRs = count_df.drop(['sample_name','phenotype_status'],axis=1).columns.values

kf = LeaveOneOut()
round = 0
for train_index,test_index in kf.split(count_df):
    round += 1
    print('Round',round)
    data = count_df.copy(deep=True)
    train_cv, test_cv = data.iloc[train_index], data.iloc[test_index]
    print(test_cv['sample_name'].tolist()[0])
    
    train_neg = train_cv[train_cv['phenotype_status']==0]
    train_pos = train_cv[train_cv['phenotype_status']==1]
    asso_TCRs = TCRs_selection_Fisher(train_neg,train_pos,TCRs,0.45)
    
    clf.fit(train_cv[asso_TCRs],train_cv['phenotype_status'])
    test_pred = clf.predict(test_cv[asso_TCRs])[0]
    test_prob = clf.predict_proba(test_cv[asso_TCRs])[:,1][0]

    sample_name.append(test_cv['sample_name'].tolist()[0])
    y_true.append(test_cv['phenotype_status'].values[0])
    y_pred.append(test_pred)
    y_proba.append(test_prob)

    
    print('y_true:',test_cv['phenotype_status'].values[0],'y_pred:',test_pred,'posterior_c1:',test_prob)
    print()

print('cv auroc:',roc_auc_score(y_true,y_proba))
print('cv log_loss:',log_loss(y_true,y_proba))

Round 1
RA47
y_true: 1 y_pred: 0 posterior_c1: 0.45694194591453435

Round 2
HC9
y_true: 0 y_pred: 1 posterior_c1: 0.9578374764538201

Round 3
RA29
y_true: 1 y_pred: 1 posterior_c1: 0.895502481831256

Round 4
RA8
y_true: 1 y_pred: 0 posterior_c1: 0.1970075685270281

Round 5
RA63
y_true: 1 y_pred: 1 posterior_c1: 0.9815459950875965

Round 6
RA33
y_true: 1 y_pred: 1 posterior_c1: 0.9926383901177553

Round 7
HC17
y_true: 0 y_pred: 1 posterior_c1: 0.7143952501760595

Round 8
HC3
y_true: 0 y_pred: 1 posterior_c1: 0.672397681057939

Round 9
RA17
y_true: 1 y_pred: 0 posterior_c1: 0.18733926158655473

Round 10
RA46
y_true: 1 y_pred: 1 posterior_c1: 0.9698294787610522

Round 11
HC6
y_true: 0 y_pred: 1 posterior_c1: 0.5633719824319456

Round 12
RA26
y_true: 1 y_pred: 1 posterior_c1: 0.9738597262350166

Round 13
RA15
y_true: 1 y_pred: 1 posterior_c1: 0.955558981784517

Round 14
RA53
y_true: 1 y_pred: 1 posterior_c1: 0.8757015592937413

Round 15
RA32
y_true: 1 y_pred: 1 posterior_c1: 0.524223941720

In [149]:
# Init lists
sample_name = []
y_true = []
y_pred = []
y_proba = [] 

TCRs = count_df.drop(['sample_name','phenotype_status'],axis=1).columns.values

kf = LeaveOneOut()
round = 0
for train_index,test_index in kf.split(count_df):
    round += 1
    print('Round',round)
    data = count_df.copy(deep=True)
    train_cv, test_cv = data.iloc[train_index], data.iloc[test_index]
    print(test_cv['sample_name'].tolist()[0])
    
    train_neg = train_cv[train_cv['phenotype_status']==0]
    train_pos = train_cv[train_cv['phenotype_status']==1]
    asso_TCRs = TCRs_selection_Fisher(train_neg,train_pos,TCRs,0.15)
    
    clf.fit(train_cv[asso_TCRs],train_cv['phenotype_status'])
    test_pred = clf.predict(test_cv[asso_TCRs])[0]
    test_prob = clf.predict_proba(test_cv[asso_TCRs])[:,1][0]

    sample_name.append(test_cv['sample_name'].tolist()[0])
    y_true.append(test_cv['phenotype_status'].values[0])
    y_pred.append(test_pred)
    y_proba.append(test_prob)

    
    print('y_true:',test_cv['phenotype_status'].values[0],'y_pred:',test_pred,'posterior_c1:',test_prob)
    print()

print('cv auroc:',roc_auc_score(y_true,y_proba))
print('cv log_loss:',log_loss(y_true,y_proba))

Round 1
RA47
y_true: 1 y_pred: 1 posterior_c1: 0.5680193965677006

Round 2
HC9
y_true: 0 y_pred: 1 posterior_c1: 0.9908236667741187

Round 3
RA29
y_true: 1 y_pred: 1 posterior_c1: 0.9347565506801512

Round 4
RA8
y_true: 1 y_pred: 1 posterior_c1: 0.5286521355361956

Round 5
RA63
y_true: 1 y_pred: 1 posterior_c1: 0.8726907627925419

Round 6
RA33
y_true: 1 y_pred: 1 posterior_c1: 0.954283059456939

Round 7
HC17
y_true: 0 y_pred: 1 posterior_c1: 0.5580162957914521

Round 8
HC3
y_true: 0 y_pred: 1 posterior_c1: 0.5580162957914521

Round 9
RA17
y_true: 1 y_pred: 1 posterior_c1: 0.5418554046934675

Round 10
RA46
y_true: 1 y_pred: 1 posterior_c1: 0.913896548744525

Round 11
HC6
y_true: 0 y_pred: 1 posterior_c1: 0.5580162957914521

Round 12
RA26
y_true: 1 y_pred: 1 posterior_c1: 0.8726907627925419

Round 13
RA15
y_true: 1 y_pred: 1 posterior_c1: 0.968472681148255

Round 14
RA53
y_true: 1 y_pred: 1 posterior_c1: 0.8398672023164385

Round 15
RA32
y_true: 1 y_pred: 1 posterior_c1: 0.52865213553619

In [ ]:
# Init lists
sample_name = []
y_true = []
y_pred = []
y_proba = [] 

TCRs = count_df.drop(['sample_name','phenotype_status'],axis=1).columns.values

kf = LeaveOneOut()
round = 0
for train_index,test_index in kf.split(count_df):
    round += 1
    print('Round',round)
    data = count_df.copy(deep=True)
    train_cv, test_cv = data.iloc[train_index], data.iloc[test_index]
    print(test_cv['sample_name'].tolist()[0])
    
    train_neg = train_cv[train_cv['phenotype_status']==0]
    train_pos = train_cv[train_cv['phenotype_status']==1]
    asso_TCRs = TCRs_selection_ttest(train_neg,train_pos,TCRs,0.02)
    print('The number of asso_TCRs',len(asso_TCRs))
    clf.fit(train_cv[asso_TCRs],train_cv['phenotype_status'])
    test_pred = clf.predict(test_cv[asso_TCRs])[0]
    test_prob = clf.predict_proba(test_cv[asso_TCRs])[:,1][0]

    sample_name.append(test_cv['sample_name'].tolist()[0])
    y_true.append(test_cv['phenotype_status'].values[0])
    y_pred.append(test_pred)
    y_proba.append(test_prob)

    
    print('y_true:',test_cv['phenotype_status'].values[0],'y_pred:',test_pred,'posterior_c1:',test_prob)
    print()

print('cv auroc:',roc_auc_score(y_true,y_proba))
print('cv log_loss:',log_loss(y_true,y_proba))

Round 1
RA47
The number of asso_TCRs 139
y_true: 1 y_pred: 1 posterior_c1: 0.6863385446949297

Round 2
HC9
The number of asso_TCRs 156
y_true: 0 y_pred: 1 posterior_c1: 0.9999080856072344

Round 3
RA29
The number of asso_TCRs 135
y_true: 1 y_pred: 1 posterior_c1: 0.8556414669338109

Round 4
RA8
The number of asso_TCRs 149
y_true: 1 y_pred: 1 posterior_c1: 0.6863385446949297

Round 5
RA63
The number of asso_TCRs 151
y_true: 1 y_pred: 1 posterior_c1: 0.8556414669338109

Round 6
RA33
The number of asso_TCRs 147
y_true: 1 y_pred: 1 posterior_c1: 0.9540150327050733

Round 7
HC17
The number of asso_TCRs 166
y_true: 0 y_pred: 1 posterior_c1: 0.8287113880411705

Round 8
HC3
The number of asso_TCRs 158
y_true: 0 y_pred: 1 posterior_c1: 0.7047442646032164

Round 9
RA17
The number of asso_TCRs 152
y_true: 1 y_pred: 1 posterior_c1: 0.6863385446949297

Round 10
RA46
The number of asso_TCRs 146
y_true: 1 y_pred: 1 posterior_c1: 0.9111754857767611

Round 11
HC6
The number of asso_TCRs 161
y_true: 0 y